In [1]:
import os
import pandas as pd

#загрузим скаченные датасеты по тегам Data Science, Data Analyst, Data Engineer, Machine Learning, MLOps,
#Аналитик данных, Инженер по данным в единый для первичной корректировки

files = [file for file in os.listdir('habr')]
df = pd.DataFrame()
for file in files:
    one = pd.read_csv('habr/' + file)
    df = pd.concat([df, one], ignore_index = True)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4517 entries, 0 to 4516
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          4506 non-null   object
 1   spec           4517 non-null   object
 2   employer       4517 non-null   object
 3   salary         1047 non-null   object
 4   level          4517 non-null   object
 5   skills         4517 non-null   object
 6   date_publish   4506 non-null   object
 7   location       2791 non-null   object
 8   working_day    2791 non-null   object
 9   remote         2791 non-null   object
 10  employer_desc  4471 non-null   object
 11  respons        4517 non-null   object
 12  compens        2123 non-null   object
 13  add_info       424 non-null    object
dtypes: object(14)
memory usage: 494.2+ KB


Пояснения к данным:

    **title** - название вакансии
    **spec** - специализация
    **employer** - название работодателя
    **salary**  - уровень з/п в рублях, другие валюты булут конвертированы по 80 руб за ед валюту
    **level** - уровень потенциального сотрудника
    **skills** - клчевые навыки, отдельно вынесены работодателем вне основного текста **ВНИМАНИЕ** в виде списка
    **date_publish** - даты публикации вакансии
    **location** - место работы
    **working_day** - полный или неполный рабочий день
    **remote** - возможность удаленной работы
    **employer_descr** - описание работодателя, много текста
    **respons** - часть описания вакансии в большенстве случаев - обязанности и требования к оструднику, опять много текста
    **сompens** - часть описания вакансии в большенстве случаев - компенсацонный пакет
    **add_info** - часть описания вакансии в большенстве случаев - доп инфо от работодателя


In [2]:
# удалим дубликаты, которые пришли по разным тегам
columns = list(df.columns)
mask = df.duplicated(subset = columns)
df_dupl = df[mask]
print('число найденных дубликатов', df_dupl.shape[0])
df = df.drop_duplicates(subset = columns)
df.shape[0]

число найденных дубликатов 1785


2732

In [3]:
# Заменим пустоты на ноль
df.replace(to_replace=[None], value=0, inplace=True)

In [ ]:
lst_to_delete = df_title[df_title['counts']==1]['unique_values'].values
for i in lst_to_delete:
    df = df.drop(df.loc[df['spec'] == i].index, axis =0)   

In [5]:
df['spec'].unique()

array(['Аналитик по данным', 'Системный аналитик', 'Антифрод аналитик',
       'Продуктовый аналитик', 'Инженер по данным', 'Учёный по данным',
       'Бэкенд разработчик', 'Администратор баз данных',
       'Бизнес-аналитик', 'Инженер по обеспечению качества',
       'Девопс инженер', 'Веб-аналитик', 'Разработчик игр',
       'Инженер по безопасности', 'Технический директор',
       'Разработчик баз данных', 'Системный инженер',
       'Десктоп разработчик', 'Инженер по автоматизации тестирования',
       'Разработчик мобильных приложений',
       'Архитектор программного обеспечения', 'Архитектор баз данных',
       'Менеджер продукта', 'Технический писатель', 'Фулстек разработчик',
       'BI-разработчик', 'Менеджер по маркетингу',
       'Инженер технической поддержки', 'Менеджер проекта',
       'Фронтенд разработчик', 'Разработчик приложений',
       'Инженер по производительности', 'SEO-специалист',
       'Системный администратор', 'UI/UX дизайнер',
       'Администратор сервер

In [6]:
# удалим ЕЩЕ раз, как-то прокрались они
lst_to_delete = ['Юрист', 'Офис менеджер','Менеджер по найму','Графический дизайнер',
                 'Арт директор','Менеджер по обслуживанию клиентов'
                ]
for i in lst_to_delete:
    df = df.drop(df.loc[df['spec'] == i].index, axis =0)

In [7]:
# поработаем с зп, обратим ее в числовой формат
df['r_salary'] = df['salary'].astype(str).apply(lambda x: x.replace('от','').replace(' ','').
                                                replace('₽','').replace('$','').replace('€','').
                                                split('до')[-1]
                                               )
df['r_salary'] = df['r_salary'].astype(int)
df['r_salary'] = df['r_salary'].apply(lambda x: x if x > 20000 else x*80 )
df['r_salary'].unique()

array([     0, 120000, 240000, 180000, 200000, 320000, 150000, 300000,
       350000, 250000, 280000,  76900, 600000, 400000, 680000, 520000,
       160000, 270000,  70000, 195000, 199000, 480000, 528000, 800000,
       536000, 940800, 360000, 440000, 100000, 130000, 140000,  60000,
       105000,  40000, 380000, 220000,  61800,  50000, 104000, 170000,
        90000, 500000, 190000, 110000,  80000,  30000, 230000, 550000,
       420000,  35000, 218000, 145200,  28000, 165000,  71000,  56000,
        55000,  46200, 115000,  38000,  24000,  64000, 206000, 144000,
        25000,  85000, 243000, 260000, 210000,  57000, 640000, 450000,
       170500, 448000, 272000], dtype=int64)

In [8]:
df['level'].unique()

array(['Старший (Senior)', 'Средний (Middle)', 'Анализ данных',
       'Защита информации', 'Системный анализ', 'Младший (Junior)',
       'Обратная разработка', 'SQL', 'T-SQL', 'Анализ требований',
       'Машинное обучение', 'SOAP', 'Управление людьми', 'Python',
       'Ведущий (Lead)', 'Java', 'Linux', 'B2C', 'C++',
       'Управление проектами', 'Интернет маркетинг', 'Swift', 'ArchiMate',
       'UML', 'BPMN', 'JavaScript', 'MySQL', 'Английский язык', 'Ansible',
       'Powershell', 'REST', 'Нагрузочное тестирование', 'Git', 'BGP',
       'Postman', 'Тестирование мобильных приложений',
       'Ручное тестирование', 'PostgreSQL', 'Kubernetes',
       'Автоматизация тестирования', 'Администрирование сетей', '.NET',
       'Стажёр (Intern)', 'Тестирование ПО', 'Golang', 'Ruby', 'C#',
       'Тестирование на проникновение', 'Многопоточность', 'Scala',
       'Архитектура предприятия', 'Microsoft SQL', 'Системная аналитика',
       'Администрирование 1С', 'Microsoft Visio', 'HTTP', 'ОО

In [9]:
# очевидно, что работодатель запонял не только уровень, но и скилы. 
# Перенесем скилы в другу колонку к скилам

level_lst = ['Старший (Senior)', 'Средний (Middle)','Младший (Junior)', 'Ведущий (Lead)', 'Стажёр (Intern)']
df['main_skill'] = df['level'].apply(lambda x: x if x not in level_lst else 'No main')
df['level'] = df['level'].apply(lambda x: x if x in level_lst else 'Not indicated')
df['skills'] = df['main_skill'].astype(str) + df['skills'].astype(str)
df = df.drop('main_skill', axis = 1)

In [10]:
df['level'].value_counts()

Средний (Middle)    1266
Старший (Senior)     808
Not indicated        341
Ведущий (Lead)       205
Младший (Junior)      71
Стажёр (Intern)       11
Name: level, dtype: int64

In [11]:
#удалим лишний кавычки и переведем в формат списка
df['skills'] = df['skills'].apply(lambda x: x.replace('[','').replace(']','').replace('No main','').replace(",", '').strip())
df['skills'] = df['skills'].astype(str)
def skills_clean(skill):
    test = []
    for i in skill.split("'"):
        if i != ' ' and i != '':
            test.append(i)
    return test
df['skills'] = df['skills'].apply(skills_clean)

In [ ]:
# переведем даты в формат дат
df['date_publish'] = pd.to_datetime(df['date_publish'])
df['date_publish'] = df['date_publish'].dt.strftime('%d-%m-%Y')

In [12]:
# удалим лишне знаки в мете работы
df['location'] = df['location'].astype(str)
df['location'] = df['location'].apply(lambda x:  x.replace ('0', 'Not indicated').replace(' • ',''))

In [13]:
df['location'].unique()

array(['Ростов-на-Дону, Ярославль, Гомель (Беларусь)', '',
       'Not indicated', 'Москва', 'Москва, Санкт-Петербург, Казань',
       'Москва, Санкт-Петербург, Ростов-на-Дону', 'Минск (Беларусь)',
       'Санкт-Петербург, Воронеж',
       'Казань, Минск (Беларусь), Нижний Новгород',
       'Санкт-Петербург, Казань, Гомель (Беларусь)', 'Санкт-Петербург',
       'Казань', 'Москва, Екатеринбург', 'Москва, Санкт-Петербург',
       'Краснодар', 'Липецк', 'Архангельск',
       'Москва, Санкт-Петербург, Нижний Новгород', 'Набережные Челны',
       'Санкт-Петербург, Екатеринбург, Казань',
       'Москва, Санкт-Петербург, Новосибирск',
       'Москва, Санкт-Петербург, Краснодар', 'Москва, Белград (Сербия)',
       'Екатеринбург', 'Киров', 'Волгоград',
       'Брянск, Смоленск, Орел-Изумруд, с.', 'Пермь, Уфа, Челябинск',
       'Лимассол (Кипр)', 'Новосибирск', 'Самара', 'Ижевск',
       'Санкт-Петербург, Тбилиси (Грузия)', 'Нижний Новгород',
       'Москва, Санкт-Петербург, Екатеринбург',
    

In [14]:
#удалим лишние знаки в рабочем графике
df['working_day'] = df['working_day'].astype(str)
df['working_day'] = df['working_day'].apply(lambda x:  x.replace ('0', 'Not indicated').replace(' • ',''))
df['working_day'].value_counts()

Полный рабочий день      1621
Not indicated            1046
Неполный рабочий день      35
Name: working_day, dtype: int64

In [15]:
# удалим лишние знаки
df['remote'] = df['remote'].astype(str)
df['remote'] = df['remote'].apply(lambda x:  x.replace ('0', 'Not indicated'))
df['remote'].value_counts()

Можно удаленно    1656
Not indicated     1046
Name: remote, dtype: int64

In [19]:
df.reset_index()

,index,title,spec,employer,salary,level,skills,date_publish,location,working_day,remote,employer_desc,respons,compens,add_info,r_salary
0,0,Data Analyst,Аналитик по данным,Aston (ex. Andersen),0,Старший (Senior),[SQL],2022-12-22T21:12:11+03:00,"Ростов-на-Дону, Ярославль, Гомель (Беларусь)",Полный рабочий день,Можно удаленно,"Аутсорсинговая компания, ориентированная на ра...",Аутсорсинговая аккредитованная IT-компания Ast...,Что мы предлагаем:\n- работу в аккредитованной...,0,0
1,1,Data analyst,Аналитик по данным,Банк ВТБ,0,Средний (Middle),"[SQL, Python, Базы данных]",2022-12-18T17:27:52+03:00,,Полный рабочий день,Можно удаленно,Российский универсальный коммерческий банк c г...,В кросс-функциональную команду аналитиков данн...,трудоустройство согласно Законодательству;конк...,Требования:\nвысшее образование;уверенные знан...,0
2,2,Data analyst,Аналитик по данным,Сбер,0,Старший (Senior),"[SQL, Python]",2022-12-19T10:27:59+03:00,Not indicated,Not indicated,Not indicated,"Крупнейшая цифровая платформа. Технобренд, объ...","Кто мы? Продуктовая команда ""Индустрии Рекламы...",0,0,0
3,3,Data Analyst (проект CVM),Аналитик по данным,Магнит,0,Средний (Middle),"[SQL, Python]",2022-12-18T12:28:00+03:00,Москва,Полный рабочий день,Можно удаленно,"Объединяем IT и ритейл, чтобы совершать самые ...",Наши задачи:\nПоддерживать и контролировать ме...,Мы предлагаем:\nГрафик работы 5/2 с гибким нач...,0,0
4,4,Data Quality analyst,Аналитик по данным,Группа «Самолет»,0,Средний (Middle),"[Git, Python, SQL]",2022-12-19T16:27:31+03:00,,Полный рабочий день,Можно удаленно,Топ-5 среди застройщиков Москвы и МО,Перед нашей командой стоят крайне амбициозные ...,Мы предлагаем:\nОфициальное трудоустройство и ...,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2697,4512,Lead Fullstack Developer,Фулстек разработчик,Aston (ex. Andersen),от 300 000 до 400 000 ₽,Ведущий (Lead),"[Java, React, JavaScript]",2022-12-20T18:08:50+03:00,Казань,Полный рабочий день,Можно удаленно,"Аутсорсинговая компания, ориентированная на ра...",Аутсорсинговая аккредитованная IT-компания Ast...,0,0,400000
2698,4513,Lead Fullstack Developer,Фулстек разработчик,Aston (ex. Andersen),от 300 000 до 400 000 ₽,Ведущий (Lead),"[Java, React, JavaScript]",2022-12-20T18:08:50+03:00,Санкт-Петербург,Полный рабочий день,Можно удаленно,"Аутсорсинговая компания, ориентированная на ра...",Аутсорсинговая аккредитованная IT-компания Ast...,0,0,400000
2699,4514,Архитектор информационной безопасности,Инженер по безопасности,Сбер,0,Старший (Senior),[Информационная безопасность],2022-12-20T14:57:03+03:00,Not indicated,Not indicated,Not indicated,"Крупнейшая цифровая платформа. Технобренд, объ...",Платформа ГосТех - единая экосистема для разра...,0,0,0
2700,4515,Java разработчик (Платформа) в Одноклассники,Бэкенд разработчик,VK,0,Старший (Senior),[Java],2022-12-19T10:25:05+03:00,Not indicated,Not indicated,Not indicated,Место встречи лучших,"Команда Платформы Одноклассников ищет коллег, ...",0,0,0


In [20]:
df.to_csv('habr/final_HABR_reworked.csv')